In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import re
from transformers import AutoTokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import datetime
import warnings
warnings.filterwarnings('ignore')

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
class cfg:
    usr_token = "<usr>"
    sys_token = "<sys>"
    bos_token = '<s>'
    eos_token = '</s>'
    mask_token = '<mask>'
    pad_token = '<pad>'
    unk_token = '<unk>'
    max_len = 256
    max_turns = 2
    pre_epochs = 0
    epochs = 5
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = 1e-4
    model_name = "skt/kogpt2-base-v2"
    labeling_type = True

tokenizer = AutoTokenizer.from_pretrained(cfg.model_name,
            bos_token=cfg.bos_token, eos_token=cfg.eos_token, unk_token=cfg.unk_token,
            pad_token=cfg.pad_token, mask_token=cfg.mask_token, model_max_length = cfg.max_len)

model = GPT2LMHeadModel.from_pretrained(cfg.model_name)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [7]:
# path = './check_point/2023-02-01-08h-18m_epoch_5_sk_labeling_True.pth' # 말은 길게 하는데 내용이 애매함.
# path = './check_point/2023-02-01-07h-03m_epoch_4_sk_labeling_True.pth' # 현재까지 best..
# path = './check_point/2023-02-01-05h-49m_epoch_3_sk_labeling_True.pth'
path = './check_point/23-02-06_17h24m_epoch5_sys.pth'

model.load_state_dict(torch.load(path)) 
model.to(cfg.device)
print('완료')

완료


In [8]:
history = []
bad_words = [tokenizer.encode(token) for token in [cfg.eos_token, cfg.unk_token, cfg.sys_token, cfg.usr_token, 'ㅋ', 'ㅠ', 'ㅠㅠ', 'ㅜㅜ', 'ㅜ', 'ㅎ', 'ㅠㅜ', 'ㅜㅠ', 'ㅋㅎ', 'ㅎㅋ', '.', '하하']]
model.to(cfg.device)

while True:
    _input = input('user >> ')
    input_sent = cfg.usr_token + _input + cfg.sys_token
    
    if _input == '끝':
        break
    if len(history) > cfg.max_turns * 2:
        history = history[2: ]
    if len(history) == 0:
        input_sent = cfg.bos_token + input_sent

    history.append(input_sent)
    
    input_ids = tokenizer.encode(' '.join(history), return_tensors="pt").to(cfg.device)
    
    with torch.no_grad():
        gen_ids = model.generate(
                        input_ids,
                        temperature=0.9,
                        top_k=3,
                        top_p=0.95,
                        num_beams=3,
                        do_sample=True,
                        max_length=1000,
                        no_repeat_ngram_size=3,
                        repetition_penalty=2.0,
                        penalty_alpha=2.0,
                        max_new_tokens=25,
                        length_penalty= 0.1,
                        early_stopping= True,
                        bos_token_id = tokenizer.convert_tokens_to_ids(cfg.sys_token),
                        eos_token_id = tokenizer.convert_tokens_to_ids(cfg.usr_token),
                        bad_words_ids = bad_words
                    )        
        
    
    generated = gen_ids[0, input_ids.shape[-1]: -1]
    generated = tokenizer.decode(generated)
    history.append(generated)

    print(f'bot  >> {generated}')

user >> 점심 먹었어?
bot  >> 응응 아직 안먹었어! 넌 뭐 먹었오?..너는 뭐 먹고싶은거 없어?
user >> 나는 비빔밥 먹고싶어!
bot  >> 비빔밥이랑 오뎅탕도 먹고싶다 _비빔밥 맛있겠다나는 콩나물 국밥 먹고
user >> 끝


In [4]:
"""
대화 함수
끝으로 대화 종료
초기화로 이전 대화 초기화

이전 대화가 history에 남아있음.
"""

history = ""
start=True
model.to(cfg.device)

while True:
    _input = input("user > ")
    
    if _input == "끝":
        break
    if _input == "초기화":
        history = ""
        continue
    if start:
        _input_word = cfg.bos_token + cfg.usr_token + _input + cfg.sys_token
        history += _input_word
        start = False
    else:
        _input_word = cfg.usr_token + _input #+ cfg.sys_token
        history += _input_word
        
    input_ids = tokenizer.encode(history, return_tensors="pt").to(cfg.device)
    
    with torch.no_grad():
        gen_ids = model.generate(
            input_ids,
            tok_k=3,
            top_p=0.92,
            num_beams=3,
            do_samples=True,
            max_length=100,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            temperature=0.9,
            max_new_tokens=20,
#             forced_bos_token_id= tokenizer.convert_tokens_to_ids(cfg.sys_token),
#             forced_eos_token_id= tokenizer.convert_tokens_to_ids(cfg.eos_token),
            bad_words_ids=[[tokenizer.convert_tokens_to_ids(cfg.sys_token)], [tokenizer.convert_tokens_to_ids('하하')]]
        )
    
    gen = tokenizer.decode(gen_ids[0])
    try:
        generated = gen[gen.rfind("<sys>")+5:gen.index("</s>")]
    except:
        generated = gen[gen.rfind("<sys>")+5:]
    history += generated
    
    print(f'Chatbot > {gen}')
    print(f'Chatbot > {generated}')

user > 운동 좋아해?


/opt/conda/lib/python3.9/site-packages/transformers/generation_utils.py:874: UserWarning: Both `max_length` and `max_new_tokens` have been set but they serve the same purpose.
  warnings.warn(


Chatbot > <s><usr> 운동 좋아해?<sys> 난 피티받구 싶어 ᄏᄏ피티는 돈내고 하는 운동!? </s>
Chatbot >  난 피티받구 싶어 ᄏᄏ피티는 돈내고 하는 운동!? 
user > 그치 돈내고 하는 운동이지 피티는 ㅋㅋ
Chatbot > <s><usr> 운동 좋아해?<sys> 난 피티받구 싶어 ᄏᄏ피티는 돈내고 하는 운동!? <usr> 그치 돈내고 하는 운동이지 피티는 ᄏᄏ아 근데 피티로 하는거 넘 비싸 ㅠㅠ ᅮ 뭔가 하고 싶은거</s>
Chatbot >  난 피티받구 싶어 ᄏᄏ피티는 돈내고 하는 운동!? <usr> 그치 돈내고 하는 운동이지 피티는 ᄏᄏ아 근데 피티로 하는거 넘 비싸 ㅠㅠ ᅮ 뭔가 하고 싶은거
user > 끝


In [281]:
"""
대화 함수
끝으로 대화 종료
초기화로 이전 대화 초기화

이전 대화가 history에 남아있음.
"""

history = ""
start=True
model.to(cfg.device)

while True:
    _input = input("user > ")
    
    if _input == "끝":
        break
    if _input == "초기화":
        history = ""
        continue
    if start:
        _input_word = cfg.bos_token + cfg.usr_token + _input + cfg.sys_token
        #_input_word =  _input
        history += _input_word
        start = False
    else:
        _input_word = cfg.usr_token + _input + cfg.sys_token
        history += _input_word
        
    input_ids = tokenizer.encode(history, return_tensors="pt").to(cfg.device)
    
    with torch.no_grad():
        gen_ids = model.generate(
            input_ids,
            top_k=5,
            top_p=0.92,
            max_length=1000,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            temperature=0.5,
            max_new_tokens=30,
            bos_token_id = tokenizer.convert_tokens_to_ids(cfg.sys_token),
            eos_token_id = tokenizer.convert_tokens_to_ids(cfg.usr_token),
#             forced_bos_token_id= tokenizer.convert_tokens_to_ids(cfg.sys_token),
#             forced_eos_token_id= tokenizer.convert_tokens_to_ids(cfg.eos_token),
            bad_word_ids=[[tokenizer.convert_tokens_to_ids(cfg.sys_token)], [tokenizer.convert_tokens_to_ids('하하')]]
        )
    
    gen = tokenizer.decode(gen_ids[0])
        
    history += generated
    
    print(f'gen > {gen}')
    print(f'Chatbot > {generated}')

user > 치킨 먹고싶다
gen > <s><usr> 치킨 먹고싶다<sys> 난 교촌치킨이 땡겨<usr>
Chatbot > 
user > 교촌치킨 좋아해?
gen > <s><usr> 치킨 먹고싶다<sys><usr> 교촌치킨 좋아해?<sys> 응응 맛있더라 ᄏᄏ<sys> 교촌에서 먹은거중에 제일 맛있는듯<sys> 교촌은 진짜 다 맛있어..<sys> 근데
Chatbot > 
user > 끝


In [263]:
history

'<s><usr>운동 좋아해?<sys>'

In [278]:
gen[len(history)+1:]

' 나는 헬스 좋아해. 너는?<sys> 나도! ᄏᄏ근데 요즘 코로나때문에 못가고 있어.<sys> 아 맞다'

In [260]:
a = '라면 먹었어<sys> 왜 라면 먹었어'
b = '라면 먹었어<sys> 왜 라면 먹었어 <usr> 그냥 배고파서!'

In [79]:
gen[86:]

'<usr>'

In [77]:
len(''.join(history))

84

In [74]:
gen[len(''.join(history)):]

'ᅲ.<usr>'

In [68]:
gen[len(''.join(history)):]

'나도<usr>'

In [70]:
''.join(history)

'<s><usr>필라테스 하러갈래?<sys>필라테스? ᄏᄏ<usr>응응 필라테스!<sys>> 필라테스가 뭐더라..? 필라테스는.. 뭔가.. ᄏ ᄏ나도'

In [66]:
history

['<s><usr>필라테스 하러갈래?<sys>',
 '필라테스? ᄏᄏ',
 '<usr>응응 필라테스!<sys>',
 '> 필라테스가 뭐더라..? 필라테스는.. 뭔가.. ᄏ ᄏ나도']

In [69]:
gen

'<s><usr> 필라테스 하러갈래?<sys> 필라테스? ᄏᄏ<usr> 응응 필라테스!<sys> 필라테스가 뭐더라..? 필라테스는.. 뭔가.. ᄏ ᄏ나도<usr>'

In [59]:
history

['<s><usr>운동 좋아해?<sys>',
 '나는 운동 좋아하긴 하는데 ᄏᄏ너는?',
 '<usr>나도 좋아해!<sys>',
 '',
 '<usr>아 진짜?<sys>',
 '']

In [44]:
tokenizer.encode('운동은 어떤 거 해봤어? 하하나는 필라테스 했었어! ᄏᄒᄏ')

[19295,
 9863,
 9122,
 9098,
 7662,
 8006,
 406,
 9078,
 8702,
 10972,
 20666,
 21422,
 9651,
 8017,
 8006,
 376,
 739,
 605,
 608,
 605]

In [58]:
tokenizer.encode('나는')

[12102]

In [57]:
tokenizer.decode(437)

'_'

In [51]:
tokenizer.decode(8702)

'하'

In [28]:
gen[len(''.join(history))+2:gen.rfind('<usr>')]

'무슨카페갈까'

In [315]:
tokenizer.decode(*gen_ids)

'<s><usr> 점심 먹었어?<sys> 아니 아직!뭐먹을지 고민중이야 ᄏᄏ배고파.. ㅠㅠ ᅮ<sys> 배고프다잉 ᅲᅮ난 이제 저녁 먹어야징 ㅠㅠ 밥이없어서 못먹는중..밥은 먹엇어? ᄒ.ᄏ<sys> 응응 밥먹'

In [303]:
tokenizer.decode(*gen_ids)

'<s><usr> 점심 먹었어?<sys> 응 먹었지. 너는?<sys> 나는 아직 안먹었어 ᄏᄏ너는 뭐 먹어?<sys> 난 오늘 점심 뭐 먹었는지 기억이 안나네 ㅠㅠ 너는 뭐 먹었는데?<sys> 나도 모르겠다 ㅠㅠ 뭐가 먹고싶은데?<sys> 음..<usr>'

In [304]:
gen_ids

tensor([[    0,     2,  9240,  7894,  9784,  8017,  8006,   406,     4, 10223,
          9784,  8017,  8263,   389, 10099,  7162,   406,     4, 12102, 11059,
          9183,  7513,  8017,  8006,   739,   605,   605, 26215, 46651, 33146,
           406,     4,  9769, 10070,  9240,  7894, 46651,  9784,  8017, 15433,
         14337,  8146,  9183,  7055,  7098,   739,   216, 10099,  7162, 46651,
          9784, 16693,   406,     4, 35704, 42138,  7182,   739,   216, 46651,
          6824, 15495,  7898,  8135,  7220,   406,     4,  9330,  9705,     2]],
       device='cuda:0')

In [231]:
gen.rfind('<usr>')

25

In [203]:
a = '<s><usr> 점심 먹었어? 응응 먹었지! <usr> 뭐 먹었어? <sys> 삼겹살 구워먹었엉ᄏᄏ <sys> 오홍ᄏᄏ 맛있겠'

In [204]:
a.rfind('<sys>')

56

In [188]:
tokenizer.get_vocab()[cfg.eos_token]

1

In [189]:
tokenizer.convert_tokens_to_ids(cfg.eos_token)

1

In [176]:
history

'<s><usr>과일 좋아하는거 있어? 귤은 맛있는데 귤은 너무 달아서 별로야 ㅠㅠ<usr> 맞아 ㅠㅠ 나는 귤이'

In [178]:
gen[gen.find(history):]

'이'

In [118]:
tokenizer.decode(*gen_ids)

'<usr> 운동하러갈래?<sys> 머하고싶어? <sys> 축구 ᄏᄏ <sys> 족구 <sys> 축구는 안땡기네 <sys> 뭐먹고싶은데 <sys> 머가땡겨 <sys> <usr>'

In [86]:
gen[gen.rfind("<sys>")+5:gen.index("</s>")]

ValueError: substring not found

In [80]:
generated

' 아니 아직 안 먹음 ᄏᄏ<usr'

In [77]:
gen[gen.rfind("<sys>")+5:gen.rfind("<usr>")]

' 아니 아직 안 먹음 ᄏᄏ'

In [69]:
generated

''

In [56]:
tokenizer.get_vocab()[cfg.eos_token]

1

In [329]:
_input = '점심 먹었어?'
input_word = cfg.bos_token + cfg.usr_token + _input + cfg.sys_token
input_ids = tokenizer.encode(input_word, return_tensors="pt").to(cfg.device)
with torch.no_grad():
    outputs = model(input_ids)

In [330]:
tokenizer.decode(*torch.argmax(outputs.logits, dim=-1))

'아데먹었어?<sys> 응 '